# Prompt Enginnering
### **Course Name:** Natural Language Processing **<font color="red">(CSC5051)</font>**




Hello, everyone.
In this tutorial, we'll explore *Prompt Engineering* techniques.

**<font color="blue">What is prompt engineering in AI?</font>**

An AI prompt is a carefully crafted instruction given to an AI model to generate a specific output. These inputs can range from text and images to videos or even music.

Prompt engineering means writing precise instructions that guide AI models like **ChatGPT** to produce specific and useful responses. It involves designing inputs that an AI can easily understand and act upon, ensuring the output is relevant and accurate.

## **The First Step: Mastering the Use of APIs**



### About API Keys
#### DeepSeek API Keys
❗ Note that you can go [Deepseek](https://platform.deepseek.com/api_keys) to get your api key. Alternatively,  youbcan go to [Siliconflow](https://cloud.siliconflow.cn/) for free credit:
- Go to the website [https://platform.deepseek.com/api_keys](https://platform.deepseek.com/api_keys).
- Setup your api key through setting the environment variable os.environ["DEEPSEEK_API_KEY"]
- Remember to update `DEEPSEEK_BASE_URL` to https://api.siliconflow.cn/v1/chat/completions when using API from SiliconFlow.

#### OpenAI API Keys
Note that we provide a key with 100 US dollars, if it is used up you need to buy the Keys yourself (it may cost you a little bit of money), here is how to buy the keys:
- Go to the website [https://eylink.cn/](https://eylink.cn/buy/7).
- Purchase a 12 RMB key (10 US dollars). (10 dollars should be enough.)
- Fill in the `OPENAI_API_KEY` below with the key you purchased.

🔅 To facilitate easier access to OpenAI's model APIs, we make use of a popular framework langchain.

In [1]:
!pip install langchain
!pip install langchain-openai
!pip install langchain-deepseek
!pip install retrying

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 14.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.0 which is incompatible.


In [2]:
import os
import time
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import random
import json
from retrying import retry
import requests


os.environ["DEEPSEEK_API_KEY"] = "sk-d887c182561c4cf98855733aa41dcaf4"
os.environ["DEEPSEEK_BASE_URL"] = "https://api.deepseek.com/v1"
deepseek_chat = ChatDeepSeek(model="deepseek-chat", temperature=1)

In [3]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", ""),
        ("user", "{input}")
    ]
)

model = ChatDeepSeek(model="deepseek-chat")

chain = prompt | model | StrOutputParser()

response = chain.invoke({"input": "Hello"})
print(response)

Hello! 👋 How can I help you today?


In [ ]:
chain = prompt | model | StrOutputParser()

response = chain.invoke({"input": "Hello"})
print(response)

Hello! How can I assist you today? 😊


## **Task 1: LLMs as a knowledgeable doctor**




The pharmacist licensure exam  is a cornerstone in the pharmacy profession, ensuring that candidates possess the requisite knowledge and skills for safe and effective practice. Its significance lies not only in validating credentials but also in safeguarding public health, enabling professional recognition, and ensuring adherence to legal and regulatory standards.

Advanced models like ChatGPT have significant potential in exam preparation, boasting an extensive knowledge base and the capability to provide in-depth explanations and clarify complex concepts. However, despite the prowess of such large models, if prompts are not designed appropriately, the information retrieved might be inaccurate or incomplete, potentially hindering success in the pharmacist exam.

In [8]:
!wget https://NLP-course-cuhksz.github.io/Assignments/Assignment1/task1/data/1.exam.json

--2025-10-24 07:18:33--  https://nlp-course-cuhksz.github.io/Assignments/Assignment1/task1/data/1.exam.json
Resolving nlp-course-cuhksz.github.io (nlp-course-cuhksz.github.io)... 185.199.111.153, 185.199.110.153, 185.199.109.153, ...
Connecting to nlp-course-cuhksz.github.io (nlp-course-cuhksz.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 86227 (84K) [application/json]
Saving to: ‘1.exam.json’

1.exam.json         100%[===================>]  84.21K  --.-KB/s    in 0.01s   

2025-10-24 07:18:33 (6.15 MB/s) - ‘1.exam.json’ saved [86227/86227]



In [9]:
import json
with open('1.exam.json') as f:
  data = json.load(f)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Data cleaner

In [10]:
tmp_data = []
for d in data:
    # 检查所有选项是否非空
    skip = False
    for k in d['option']:
        if d['option'][k].replace(' ', '') == '':
            skip = True
            break  # 发现一个空选项即可跳过

    # 只有当所有选项都有内容时才添加
    if not skip:
        tmp_data.append(d)

data = tmp_data
print(len(data))

69


In [ ]:
data[0]

{'question': '112. 患者，男，70岁，因抑郁症长期服⽤舍曲林。新诊断为帕⾦森病，宜选⽤的药物有（）。',
 'option': {'A': '司来吉兰', 'B': '普拉克索', 'C': '⾦刚烷胺', 'D': '吡贝地尔', 'E': '恩他卡朋'},
 'analysis': '上述题⼲中患者使⽤的抑郁药舍曲林是选择性5-羟⾊胺再摄取抑制剂，司来吉兰避免与选择性5-羟⾊胺再摄取抑制剂合⽤，因此答案选BCDE。',
 'answer': 'BCDE',
 'question_type': '多项选择题',
 'source': '2021年执业药师职业资格考试《药学综合知识与技能》'}

### Original prompt

In [14]:
import re
from tqdm import tqdm

def get_query(da,template_func):
  da['options'] = '\n'.join([f"{k}:{v}" for k,v in da['option'].items()])
  return template_func.format_map(da)

def get_ans(ans):
    match = re.findall(r'.*?([A-E]+(?:[、, ]+[A-E]+)*)', ans)
    if match:
        last_match = match[-1]
        return ''.join(re.split(r'[、, ，]+', last_match))
    return ''

original_prompt = """Please answer the multiple choice questions below, please direct the correct answer option and do not output anything else.
{question}
{options}"""

example_prompt = get_query(data[0])
print(example_prompt)

TypeError: get_query() missing 1 required positional argument: 'template_func'

In [ ]:


correct_num = 0
total_num = 0
for da in tqdm(data):
  da['deepseek_ans'] =  chain.invoke(get_query(da))
  # print(da['deepseek_ans'])
  if get_ans(da['deepseek_ans']) == da['answer']:
    correct_num += 1
  total_num += 1
print(f"模型准确率: {correct_num/total_num:.2%}")


100%|██████████| 69/69 [01:50<00:00,  1.60s/it]

模型准确率: 86.96%


### My prompt with 3-shot

In [ ]:
# 计算做题准确率
system_prompt = """你是一位医学专家。请回答给定的医学问题。仅输出正确的答案选项。
"""

user_template = '''
问题：{question}
问题类型：{question_type}
选项：{options}
答案："
'''

few_shot_template = '''
-- 示例 --
问题：{question}
问题类型：{question_type}
选项：{options}
分析：{analysis}
答案：{answer}

'''
import re
from tqdm import tqdm
import random
def get_ans(ans):
    match = re.findall(r'.*?([A-E]+(?:[、, ]+[A-E]+)*)', ans)
    if match:
        last_match = match[-1]
        return ''.join(re.split(r'[、, ，]+', last_match))
    return ''

def get_query(da,template_func=your_prompt):
  da['options'] = '\n'.join([f"{k}:{v}" for k,v in da['option'].items()])
  return template_func.format_map(da)

correct_num = 0
total_num = 0
print_flag = True
for i, da in tqdm(enumerate(data)):
  instruct_prompt = system_prompt
  user_prompt = get_query(da,template_func=user_template)
  few_shots = random.sample(data[:i] + data[i+1:],3) if i != len(data) - 1 else random.sample(data[:i],3)
  prompt = instruct_prompt + '\n'.join([get_query(few_shot,few_shot_template) for few_shot in few_shots]) + user_prompt
  # print(prompt)
  da['deepseek_ans'] = chain.invoke(prompt)
  if get_ans(da['deepseek_ans']) == da['answer']:
    correct_num += 1
    if print_flag:
      print_flag=False

  else:
    # print(da)
    # print('*'*20)
    pass
  total_num += 1

print(f"模型准确率: {correct_num/total_num:.2%}")


69it [01:56,  1.69s/it]

模型准确率: 92.75%


<font color="blue">You need to optimize the prompt to improve the performance (accuracy) of large language models (LLMs).</font>

### My prompt with extra_info




In [ ]:
# 计算做题准确率
system_prompt = """你是一位医学专家。请根据提供的医学内容、问题来源和题目类型回答以下问题。

按照指定的题目类型回答医学问题并仅输出正确的答案选项。
"""

user_template = '''
问题：{question}
问题类型：{question_type}
选项：{options}
回答：{analy}"
'''

import re
from tqdm import tqdm
import random
def get_ans(ans):
    match = re.findall(r'.*?([A-E]+(?:[、, ]+[A-E]+)*)', ans)
    if match:
        last_match = match[-1]
        return ''.join(re.split(r'[、, ，]+', last_match))
    return ''

def get_query(da,template_func=your_prompt):
  da['options'] = '\n'.join([f"{k}:{v}" for k,v in da['option'].items()])
  return template_func.format_map(da)

correct_num = 0
total_num = 0
print_flag = True
for da in tqdm(data):
  # print(da)
  result = re.sub(r'[A-E]', '', da['analysis'])
  da['analy'] = result
  # print(result)
  instruct_prompt = system_prompt
  user_prompt = get_query(da,template_func=user_template)
  prompt = instruct_prompt + user_prompt
  # print(prompt)
  da['deepseek_ans'] = chain.invoke(prompt)

  # 一次性替换所有A-E字母,但是保留分析

  if get_ans(da['deepseek_ans']) == da['answer']:
    correct_num += 1
    if print_flag:
      # print(prompt)
      # print(da['deepseek_ans'])
      print_flag=False

  else:
    # print(da)
    # print('*'*20)
    pass
  total_num += 1

print(f"模型准确率: {correct_num/total_num:.2%}")


100%|██████████| 69/69 [01:41<00:00,  1.46s/it]

模型准确率: 94.20%


### My prompt with CoT

In [ ]:
model_cot = ChatDeepSeek(model="deepseek-reasoner", temperature=1)
chain_cot = prompt | model_cot | StrOutputParser()

In [ ]:
# 计算做题准确率
your_prompt = """你是一位医学专家。请回答给定的医学问题。仅输出正确的答案选项。
{question}
{options}"""

import re
from tqdm import tqdm

def get_query(da,template_func=your_prompt):
  da['options'] = '\n'.join([f"{k}:{v}" for k,v in da['option'].items()])
  return template_func.format_map(da)

def get_ans(ans):
    match = re.findall(r'.*?([A-E]+(?:[、, ]+[A-E]+)*)', ans)
    if match:
        last_match = match[-1]
        return ''.join(re.split(r'[、, ，]+', last_match))
    return ''

correct_num = 0
total_num = 0
for da in tqdm(data):
  da['deepseek_ans'] =  chain_cot.invoke(get_query(da))
  print(da['deepseek_ans'])
  if get_ans(da['deepseek_ans']) == da['answer']:
    correct_num += 1
  total_num += 1
print(f"模型准确率: {correct_num/total_num:.2%}")


  1%|▏         | 1/69 [01:12<1:22:02, 72.40s/it]

B、C、D


  3%|▎         | 2/69 [01:30<44:52, 40.19s/it]  

D


  4%|▍         | 3/69 [01:44<31:30, 28.64s/it]

...

B


 99%|█████████▊| 68/69 [29:36<00:18, 18.39s/it]

B


100%|██████████| 69/69 [29:59<00:00, 26.08s/it]

E
模型准确率: 95.65%


### My prompt with work flow(Agent)

In [5]:
# Agent building
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", ""),
        ("user", "{input}")
    ]
)

model_hint = ChatDeepSeek(model="deepseek-chat")

chain_hint = prompt | model_hint | StrOutputParser()

model = ChatDeepSeek(model="deepseek-chat")

chain = prompt | model | StrOutputParser()

In [16]:
# 计算做题准确率
system_prompt_hint = """你是一位医学教育专家。请根据学生提供的问题和选项概念，**简明**地给学生一些知识点概念提示，注意不要告诉学生答案
"""

user_template_hint = '''
问题：{question}
选项：{options}
提示："
'''

In [18]:
# 计算做题准确率
system_prompt = """你是一位医学生，按照指定的题目类型回答医学问题，做大钱，请仔细阅读给定的题目提示，仅输出正确的答案选项。
"""

user_template = '''
问题：{question}
问题类型：{question_type}
选项：{options}
题目提示： {knowledge}
'''

import re
from tqdm import tqdm
import random
def get_ans(ans):
    match = re.findall(r'.*?([A-E]+(?:[、, ]+[A-E]+)*)', ans)
    if match:
        last_match = match[-1]
        return ''.join(re.split(r'[、, ，]+', last_match))
    return ''

def get_query(da,template_func):
  da['options'] = '\n'.join([f"{k}:{v}" for k,v in da['option'].items()])
  return template_func.format_map(da)

correct_num = 0
total_num = 0
print_flag = True
for da in tqdm(data):
  knowledge_prompt = system_prompt_hint + get_query(da,template_func=user_template_hint)
  # print(knowledge_prompt)
  da['knowledge'] = chain_hint.invoke(knowledge_prompt)
  # print(da['knowledge'])


  instruct_prompt = system_prompt
  user_prompt = get_query(da,template_func=user_template)
  prompt = instruct_prompt + user_prompt
  print(prompt)
  da['deepseek_ans'] = chain.invoke(prompt)
  print(da['deepseek_ans'])
  if get_ans(da['deepseek_ans']) == da['answer']:
    correct_num += 1
    if print_flag:
      print_flag=False

  else:
    pass
  total_num += 1

print(f"模型准确率: {correct_num/total_num:.2%}")


  0%|          | 0/69 [00:00<?, ?it/s]

你是一位医学生，按照指定的题目类型回答医学问题，做大钱，请仔细阅读给定的题目提示，仅输出正确的答案选项。

问题：112. 患者，男，70岁，因抑郁症长期服⽤舍曲林。新诊断为帕⾦森病，宜选⽤的药物有（）。
问题类型：多项选择题
选项：A:司来吉兰
B:普拉克索
C:⾦刚烷胺
D:吡贝地尔
E:恩他卡朋
题目提示： 好的，同学。我们一起来梳理一下这道题的关键知识点。

这道题的核心是**药物相互作用**，特别是要避免可能引发严重不良反应的联合用药。

**关键概念提示：**

1.  **关注基础疾病与用药**：患者有**抑郁症**，正在长期服用**舍曲林**。舍曲林属于SSRI类抗抑郁药。
2.  **牢记药物相互作用禁忌**：在抗帕金森病药物中，有一类药物需要特别警惕与SSRI类抗抑郁药合用，因为这会显著增加发生一种叫做 **“5-羟色胺综合征”** 的风险。这是一种潜在危及生命的严重状况。
3.  **识别高风险药物**：你需要回忆或判断，在给出的抗帕金森病选项中，哪一种药物的作用机制会直接影响5-羟色胺系统，从而与舍曲林产生叠加效应，触发上述风险。

请基于以上几点，回顾各类抗帕金森病药物的作用机制，特别是它们与神经递质系统的关系，从而做出判断。



  1%|▏         | 1/69 [00:24<27:21, 24.14s/it]

正确答案是：**B, C, D, E**

**解析：**

这道题的核心考点是避免**5-羟色胺综合征**的风险。

1.  **风险分析**：患者长期服用**舍曲林**，这是一种选择性5-羟色胺再摄取抑制剂。如果同时使用能增加5-羟色胺活性的其他药物，会导致5-羟色胺在突触间隙过度累积，从而引发5-羟色胺综合征。
2.  **关键药物判断**：在抗帕金森病药物中，**司来吉兰**在较高剂量（>10mg/日）时，会失去B型单胺氧化酶的选择性，同时抑制MAO-A和MAO-B。抑制MAO-A会阻止5-羟色胺的分解，与舍曲林合用将极大地增加5-羟色胺综合征的风险。因此，**A. 司来吉兰是禁用的**。
3.  **安全药物选择**：
    *   **B. 普拉克索**：多巴胺受体激动剂，不直接影响5-羟色胺系统。
    *   **C. 金刚烷胺**：促进多巴胺释放，也可拮抗NMDA受体，不直接影响5-羟色胺系统。
    *   **D. 吡贝地尔**：多巴胺受体激动剂，不直接影响5-羟色胺系统。
    *   **E. 恩他卡朋**：儿茶酚-O-甲基转移酶抑制剂，通过抑制左旋多巴在外周的代谢来增加其入脑量，不直接影响5-羟色胺系统。

因此，为避免与舍曲林发生严重的药物相互作用，该患者应选用**普拉克索、金刚烷胺、吡贝地尔、恩他卡朋**。
你是一位医学生，按照指定的题目类型回答医学问题，做大钱，请仔细阅读给定的题目提示，仅输出正确的答案选项。

问题：2215. 可⽤于治疗蛔⾍病、蛲⾍病、钩⾍病和鞭⾍病的⼴谱驱⾍药物是（）。
问题类型：最佳选择题
选项：A:甲硝唑
B:氯硝柳胺
C:⼄胺嗪
D:甲苯咪唑
E:三氯苯达唑
题目提示： 好的，我们一起来回顾几个关键概念：

1.  **“广谱驱肠虫药”**：这个术语指的是能够有效对抗多种不同种类肠道蠕虫的药物。
2.  **药物与虫谱对应关系**：
    *   思考一下，哪些药物主要通过干扰寄生虫的能量代谢（如抑制葡萄糖摄取）来发挥作用？
    *   回忆一下，选项中哪种药物对线虫类（如蛔虫、蛲虫、钩虫、鞭虫）的成虫和幼虫都有较好的驱杀效果？
3.  **排除法**：
    *   可以想想哪些药物主要用于治疗**原虫**感染（如阿米巴、滴虫）。
    *   哪些药物主要用于治疗**绦虫**感染。


  3%|▎         | 2/69 [00:34<17:53, 16.03s/it]

...

B:硫酸阿托品眼用凝胶
你是一位医学生，按照指定的题目类型回答医学问题，做大钱，请仔细阅读给定的题目提示，仅输出正确的答案选项。

问题：72. 多数脂溶性⼩分⼦类药物的吸收机制是（）。
问题类型：配伍选择题
选项：A:膜孔滤过
B:简单扩散
C:膜动转运
D:易化扩散
E:主动转运
题目提示： 好的，我们来梳理一下相关概念。

**核心概念提示：**

1.  **药物的跨膜转运方式**主要有这几种，它们的特点各不相同。
2.  **“脂溶性”** 是一个关键属性。请思考：细胞膜的主要化学组成是什么？这种组成对什么样的物质通透性高？
3.  **“简单扩散”** 的核心驱动力是膜两侧的**浓度差**。这个过程不消耗能量，也不需要载体，顺浓度梯度进行。
4.  可以对比思考其他选项：
    *   **主动转运**：需要载体和能量，可以逆浓度梯度进行。
    *   **易化扩散**：需要载体，但不消耗能量，顺浓度梯度进行。
    *   **膜孔滤过**：主要适用于哪些物质？（如水、小分子水溶性物质）
    *   **膜动转运**：主要适用于大分子物质。

请根据这些概念的特点，结合“脂溶性小分子”的性质，判断其最可能的吸收路径。



 99%|█████████▊| 68/69 [14:21<00:12, 12.38s/it]

B:简单扩散
你是一位医学生，按照指定的题目类型回答医学问题，做大钱，请仔细阅读给定的题目提示，仅输出正确的答案选项。

问题：48. 患者，男，54岁，因痛风服⽤⾮布司他，为增强降尿酸效果，可同时使⽤的药物是（）。
问题类型：配伍选择题
选项：A:阿托伐他汀
B:甲巯咪唑
C:丙硫氧嘧啶
D:氢氯噻嗪
E:碳酸氢钠
题目提示： 好的，同学。我们一起来梳理一下这道题涉及的关键概念。

**核心知识点提示：**

1.  **治疗目标**：本题的核心是“增强降尿酸效果”。你需要回想，痛风的治疗除了直接抑制尿酸生成（如非布司他的作用）或促进尿酸排泄外，还有哪些辅助手段可以帮助降低尿酸水平或促进其溶解。

2.  **尿酸的性质**：尿酸的溶解度与体液环境的酸碱度（pH值）密切相关。请思考，在什么样的体液环境中，尿酸更容易形成结晶并沉积？反之，如何改变环境可以增加其溶解度，从而利于通过尿液排出？

3.  **药物作用机制辨析**：
    *   选项A主要影响血脂代谢。
    *   选项B和C主要用于治疗甲状腺功能亢进。
    *   选项D（氢氯噻嗪）是一种利尿剂，请注意思考它对血尿酸水平可能产生的已知影响（是升高还是降低？）。

4.  **辅助治疗策略**：在痛风的管理中，除了主要的降尿酸药物，临床上常会使用一种安全的药物来碱化尿液，作为辅助治疗。

希望这些概念提示能帮助你串联起相关知识，做出自己的判断。



100%|██████████| 69/69 [14:33<00:00, 12.66s/it]

E:碳酸氢钠
模型准确率: 88.41%
